# Spark Architecture

### Spark Cluster, Spark Execution

Spark cluster parable of classroom of teacher & students
- each table of students is like an executor & each student is a "core"
    - Each "core" is given an individual task.

Scenario 2: count total pieces in "candy bags"
- stage1: local count, each partition gets distributed to a core on a different executor.
    - driver takes result of which ever executor finishes first, then the rest of the executors commit their results after
    - Then stage1 is complete
- stage2: global count, another executor fetches all the counts from each executor after all the counts complete. These results get passed to the driver, stage2 is then complete.

//


### Shuffling & Caching

groupBy triggers a _Wide operation_. Other wide transformations include:
- `distinct, sort, join`

_Narrow transformations_ include:
- `select, filter, cast, union`

Narrow transformations = when the data is required to compute the recs in a single partition that all reside in at most 1 partition of the parent rdd.

Wide transformations = when the data is required to compute the records in a single partition that may reside in many partitions of the parent rdd.

A shuffle introduces(i.e. demarcates) stage boundaries, shuffles happen when a wide transformation happens.
Each shuffle requires a shuffle read (from disk), and a shuffle write (to disk).
1st, shuffle writes write to disk so that all subsequent shuffle reads can read those files. Shuffle writes only happen once.


//

--------------------------------------------------------------------------------

# Query Optimization

### Query Optimization

We can turn on the Adaptive Query Execution to improve the logical plan and physical plan.

//

### Demo notes

Shown an example of how a cache could accidentally block a predicate pushdown.

//

--------------------------------------------------------------------------------

# Partitioning

### Transcript

   **~~ 0sec, What we will Cover ~~**
Let's discuss partitioning. In this lesson you will understand
- the relationship between partitions and slots & cores.
- You'll configure default shuffle partitions,
- describe repartition & coalesce,
- match the number of partitions to the number of slots & cores,
- and you'll describe dynamic coalescing of shuffle partitions in AQE.


   **~~ 28sec, Cores and Slots ~~**
The spark api uses the term "core", meaning a thread available for parallel execution.

Here we refer to it as a slot to avoid confusion with the number of cores in the underlying CPU.
To which there isn't necessarily an equal number.

In most cases, if you created a cluster, you should know how many cores you have.
However, to check programmatically, you can use:
**`spark.sparkContext.defaultParallelism`**


   ~~ 1m2s, Cores in Cluster ~~
For operations like parallelize with no parent RDDs, it depends on the cluster manager.
In local mode, you'll have a number of cores on the local machine.
Mesos fine grain mode 8 and others have a total number of cores on all executor nodes or 2,
whichever is larger.


   **~~ 1m21s, Partitions of Data ~~**
**`df.rdd.getNumPartitions()`**
The 2nd half of this question is how many partitions of data do I have?
With that we have 2 subsequent questions:
1. Why do I have that many?
2. And what is a partition?

A partition is a small piece of the total dataset. The action or state of dividing or being divided into parts.
If our goal is to process all our data, say 1million records in parallel, we need to divide that data up.
If I have 8 slots for parallel execution, it would stand to reason that I want:
 1,000,000/8 ...or 125,000 records per partition.

Back on that 1st question, we can answer it by running the following command:
 **`df.rdd.getNumPartitions()`**
  which takes the initial dataframe, converts it to an RDD, then asks the RDD for the number of partitions.

It is not coincidental that we have 8 slots and 8 partitions.
In Spark 2.0 a lot of optimizations had been added to the readers.
Namely the readers looked at the number of slots, the size of the data,
and made a best guess at the number of partitions that should be created.

You could actually double the size of the data several times over in Spark
and it would still read in only 8 partitions.
Eventually it will get so big that Spark will forego optimization and read it in as 10 partitions (in that case).

8 partitions and 8 slots is just too easy, let's read in another copy of the same data.
A parquet file that was saved in 5 partitions.
This gives us an excuse to reason about the relationship between slots and partitions.

   **~~ 3m08s, Repartition a DataFrame ~~**
The key difference between the 2 are
- Coalesce N is a narrow transformation and can only be used to reduce the number of partitions.
- Repartition N is a wide transformation and can be used to increase or decrease the number of partitions.

4m15s: In our case we need to go from 5 partitions up to 8 partitions. Our only option here is repartition.
coalesce is faster, but can result in uneven partition sizes.
repartition is slower because it requires a shuffle, but results in more even partition sizes.

   **~~ 4m50s, Make the # of partitions a multiple of the # of cores ~~**
Let's make sure we're using every slot and core.
With very few exceptions you always want the number of partitions to be a multiple of the number of slots.
    e.g. with 4 slots you want 4, 8, 12, 16, 20, 24, 28, 32, etc partitions.
That way every slot is used. That is, every slot is being assigned a task.
With 5 partitions and 8 slots we are underutilizing 3 of the 8 slots.
With 9 partitions and 8 slots we just guaranteed that our job will take 2 times as long as it may need to.
For example: 10secs to process the first 8 partitions,
    then as soon as the first 8 is done, another 10 seconds to process that last partition.

5m28s: you might be asking should I use more or less partitions?
As a general guideline, it is advised that each partition when cached, is roughly around 200MB(on disc).
Size on the disc though is not a good gauge e.g. CSVs on large on disk, but small in RAM.
Consider string "12345" which is 10 bytes on disk, but the int 12345 is 4 bytes in RAM.
Parquet files are highly compressed, so they are small on disk, but when uncompressed they are large in RAM.

6m5s: on an executor with a reduced amount of RAM you might need to lower that.
For example at 8 partitions, corresponding to our max number of slots, and 200MB per partition,
that will use roughly 1.5GB. If you have transformations that will balloon that data size,
such as NLP, you are sure to run into problems.

6m35s: a question might arise, if I read my data and it comes in at 10 partitions.
Should I decrease the number of partitions to 8? Or increase the number of partitions to 16?
The answer is, it depends on the size of each partition.
- we'll read the data in, cache it, look at the size of every partition,
- if you're over 200MB, consider increasing the number of partitions to 16(to make each partition roughly 100MB).
- The goal will always be to use as few partitions as possible, while maintaining at least 1 times the number of slots.


   **~~ 7m21s, Default Shuffle Partitions ~~**
Wide operations have to shuffle the data, once the data is shuffled it has to be re-partitioned.
Unlike repartition and coalesce, we did not specify how many partitions to use.
The problem is the number of partitions we ended up with.
Besides looking at the number of tasks in the final stage, we can simply print out the number of partitions.

The default partition size(after a shuffle) is 200MB.
This is based on real world experience of Apache Spark Engineers.

8m8s: for now we can tweak it with a configuration value:
`spark.conf.get("spark.sql.shuffle.partitions")`
`spark.conf.set("spark.sql.shuffle.partitions", "8")`
We can change the config setting with this command.

   **~~ 8m26s, Partitioning Guidelines ❤️ ~~**
Always err on the side of too many small partitions, than too few large partitions.
With this rule in mind,
- target never letting partition size increase above 200MB per 8GB of slot total memory. A 1 to 40 ratio.
- For small data, target 3 partitions per core/slot.
- Read and Shuffle tasks should complete in less than 10 seconds on average.
- And the target, a medium partition size of approximately 200MB as a starting point.

9m1s: Lastly, realize that there is almost always skew in real world datasets,
which means even though you target a 200MB partition it is likely several
of your partitions will be 2 or more times larger than the 200MB target.
And we want to make sure these tasks finish in less than 10 seconds as well whenever possible.

9m25s: Shuffles are often the most expensive operation in a Spark job
and as such write/right sizing the shuffle partitions are the most crucial.
The default shuffle partitions are set to 200,
meaning that any shuffled dataset greater than 40GB will violate our maximum shuffle partition size of 200MB.
Sizing shuffle partitions is all about knowing 2 key variables:
- the amount of data coming into the larger¿ shuffle stage of an action so for example, across all the jobs in the action
- And the target size of each shuffle partition.

... a good target size will still follow our 1 to 40 ratio of partition size, executor slot memory
for this example we will use 200MB assuming an 8GB total slot memory
`P` is our partition target size of 200MB
`I` is our largest shuffle stage input and that is going to be 4TB
The equation here that we come up with is `I / P`, our shuffle partition count.
e.g.
    4TB / 200MB = 20,000 shuffle partition count
so we'd set spark.conf.set("spark.sql.shuffle.partitions", "20000")
which stands for our shuffle partition count.

   **~~ 10m44s, 3.0 Adaptive Query Execution ~~**
Dynamically coalescing shuffle partitions.
when running queries in Spark to deal with very large data,
shuffle usually has a very important impact on query performance among other things.
Shuffle is a very expensive operator as it needs to move data across the network
so that is re-distributed in a way required by downstream operators.
One key property of shuffle is the number of partitions.
The best number of partitions is data dependent, yet data sizes may differ vastly from stage to stage, query to query.
Making this number hard to tune.

If there are too few partitions then the data size of each partition may be very large,
and the tasks to process these large partitions may need to spill data to disk.
e.g. when we sort or there is an aggregate involved.
And as a result they will slow down the query.

If there are too many partitions, then the data size of each partition may be very small,
and there will be a lot of small network data fetches to read the shuffle blocks.
which can also slowdown the query because of the inefficient I/O pattern.

Having a large number of tasks also puts more burden on the spark task scheduler.
To solve this problem we can set a relatively large number of partitions,
at the beginning. Then combine adjacent small partitions into bigger partitions
at runtime by looking at shuffle file statistics.
so for example, let's say we're running the query "select max(i) from table group by j"
the input data table is rather small so there are only 2 partitions before grouping.
The initial shuffle partition number is set 5 so after local grouping
the "partition-ally" grouped data is shuffled into 5 partitions.
Without AQE, spark will start 5 tasks to do the final aggregation.
However, there are 3 very small partitions here and it would be a waste to start a separate task for each of them.


   ~~ ?m??s, Section Title ~~
_

   ~~ ?m??s, Section Title ~~
_




//

### Partitioning

Understand the relationship between partitions and slots & cores
configure default shuffle partitions
describe repartition & coalesce
match number of partitions to slots & cores
describe dynamic coalescing of shuffle partitions in AQE

In Spark, a "core" is a thread available for parallel execution.
We refer to them interchangably as slots to avoid confusion w/cores from the CPU.
`spark.sparkContext.defaultParallelism`

To get the number of partitions
`df.rdd.getNumPartitions()`

Let's say we have a pq file that was saved in 5 partitions with 8 available slots
We have 2 options to *repartition*: .coalesce(N) or .repartition(N)
pro's & con's: shuffle vs non-shuffle, even re-distribution, decrease-only vs increase-decrease

We generally want the number of partitions to a multiple of the number of available slots
e.g. if we have 4 slots ideally we should have 8 or 12 or 16... partitions

A **very general guideline** is to have each partition be roughly about 200MB... ballpark

On an executor with a reduced amount of RAM, we might need to lower the 200MB estimate. 
e.g. at 8 partitions corresponding to 4 slots, we would use close to 1.5GB 
If there are a lot of transformations that balloon each partition size, we will have problems.
So when there are a lot of transformations that balloon each partition size making the initial 
partition size, say 50MB, would probably be better. 

6m30s, Matching the number of partitions to slots
If there are 8 slots and 10 partitions, should we increase to 16 partitions or decrease to 8 partitions?
... Answer: 


10m45s, Adaptive Queary Execution



//

### Demo notes


//

### Lab notes


//